In [88]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Conv2DTranspose,BatchNormalization,AveragePooling2D,ZeroPadding2D 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
import tensorflow
import glob
from tqdm import tqdm
import warnings;
warnings.filterwarnings('ignore')
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [32]:
from time import time
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras import models
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
# load an image from file
import sys
from numpy import load
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from matplotlib.image import imread
from numpy import expand_dims

import pickle
from tensorflow.keras.preprocessing.image import array_to_img

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from numpy import expand_dims
import numpy as np
import os

In [ ]:
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    #preprocessing_function=preprocess_input,
    rescale=1/255,
    rotation_range=60,
    width_shift_range=20,
    height_shift_range=20,
    brightness_range=(.6,1.4),
    shear_range=0.8,
    zoom_range=[.5,1.9],
    channel_shift_range=.7,
    fill_mode="wrap",
    horizontal_flip=True)

In [ ]:
true_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255)

In [153]:
IMG_SIZE=56

In [ ]:
# setup generator
train_generator = datagen.flow_from_directory(
        directory=r"C:\Users\dedunn\Desktop\full_aerin",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical')

In [ ]:
true_generator = true_datagen.flow_from_directory(
        directory=r"C:\Users\dedunn\Desktop\full_aerin",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical')

In [ ]:
valid_generator =datagen.flow_from_directory(
        directory=r"C:\Users\dedunn\Desktop\full_aerin_test",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical')

In [ ]:
true_valid_generator =true_datagen.flow_from_directory(
        directory=r"C:\Users\dedunn\Desktop\full_aerin_test",
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical')

In [ ]:
for features, label in training_data:   
    X.append(features)
    y.append(label)
#X=np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [ ]:
X=X.reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [4]:
cat=os.listdir(r"C:\Users\dedunn\Desktop\full_aerin")

In [5]:
test=load_img(r"C:\Users\dedunn\Desktop\full_aerin\AERIN_AM_MOODBOARD_LR.jpg\AERIN_AM_MOODBOARD_LR.jpg")

In [154]:
training_data=[]

In [155]:
for image in range(len(cat)):
    test=(r"C:\Users\dedunn\Desktop\full_aerin\{}\{}").format(cat[image],cat[image])
    #test=load_img(r"C:\Users\dedunn\Desktop\full_aerin\{}\{}").format(cat[image],cat[image])
    img_array = cv2.imread(test,cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)).astype('uint8')
    test_image = new_array[...,::-1]
    #test=load_img(test)
    #dims=expand_dims(test,0)
    #dims=cv2.resize(dims,(IMG_SIZE,IMG_SIZE)).astype('uint8')
    training_data.append(test_image)

In [ ]:
type(training_data)

In [152]:
array_to_img(training_data[3].reshape(28,28,1))

In [156]:
X=np.array(training_data)

In [157]:
X=X.reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [ ]:
X[0].shape[1:]

In [ ]:
DATADIR=r"C:\Users\dedunn\Desktop\full_aerin"
CATEGORIES=cat

In [ ]:
def create_training_data():
    for category in CATEGORIES:
        path =  os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
                #img_array=img_to_array(img)
                #img_array=expand_dims(img_array, 0)
                new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)).astype('uint8')
                test_image = img_array[...,::-1]
                training_data.append([test_image,class_num])
            except Exception as e:
                pass
create_training_data()
#cv2.IMREAD_GRAYSCALE

In [123]:
Input_img = tensorflow.keras.Input(shape=(28, 28, 1),dtype='float32')  
    
#encoding architecture
x1 = Conv2D(256, (3, 3), activation='relu', padding='same')(Input_img)
x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x1)
x3 = MaxPool2D( (2, 2))(x2)
encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(x3)

# decoding architecture
x4 = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x5 = UpSampling2D((2, 2))(x4)
x6 = Conv2D(128, (3, 3), activation='relu', padding='same')(x5)
x7 = Conv2D(256, (3, 3), activation='relu', padding='same')(x6)
decoded = Conv2D(1, (3, 3),activation='sigmoid', padding='same')(x7)

autoencoder = Model([Input_img, decoded])
autoencoder.compile(optimizer='adam', loss='mse')

In [125]:
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=4, verbose=1, mode='auto')

a_e = autoencoder.fit(X,X,
            epochs=50,
            #shuffle=True,
            steps_per_epoch=44,
            validation_data=(X, X))
            #callbacks=[early_stopper])

ValueError: ('Error when checking model target: expected no data, but got:', array([[[[123],
         [ 52],
         [101],
         ...,
         [131],
         [117],
         [ 54]],

        [[ 55],
         [107],
         [ 84],
         ...,
         [133],
         [ 97],
         [ 83]],

        [[ 50],
         [ 48],
         [ 77],
         ...,
         [126],
         [127],
         [ 77]],

        ...,

        [[215],
         [ 97],
         [ 71],
         ...,
         [188],
         [174],
         [196]],

        [[214],
         [178],
         [158],
         ...,
         [200],
         [189],
         [191]],

        [[154],
         [180],
         [154],
         ...,
         [188],
         [189],
         [166]]],


       [[[200],
         [108],
         [ 62],
         ...,
         [ 11],
         [235],
         [183]],

        [[223],
         [220],
         [209],
         ...,
         [229],
         [229],
         [175]],

        [[ 

In [26]:
def build_autoencoder(img_shape, code_size):
    # The encoder
    encoder = Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    encoder.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    encoder.add(MaxPool2D( (2, 2)))
    #encoder.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))

    # The decoder
    decoder = Sequential()
    decoder.add(InputLayer((code_size)))
    #decoder.add(Reshape(img_shape))
    decoder.add(UpSampling2D(interpolation='nearest'))
    decoder.add(Reshape(img_shape))
    decoder.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    decoder.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    #decoder.add(UpSampling2D((2,2)))
    #decoder.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    decoder.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    decoder.add(Flatten())
    #decoder.add(UpSampling2D(2,2))
    decoder.add(Dense(np.prod(img_shape))) # np.prod(img_shape) is the same as 32*32*3, it's more generic than saying 3072
    decoder.add(Reshape(img_shape))
    #decoder.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))
    return encoder, decoder

In [ ]:
print(encoder.summary())

In [ ]:
print(decoder.summary())

In [27]:
IMG_SHAPE = X.shape[1:]
encoder, decoder = build_autoencoder(IMG_SHAPE, 32)

inp = Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

print(autoencoder.summary())

ValueError: Input 0 of layer up_sampling2d_4 is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: [None, 32]

In [28]:
history = autoencoder.fit(x=X, y=X, epochs=20000,
                validation_data=[X, X])

Train on 44 samples, validate on 44 samples
Epoch 1/20000
32/44 [====================>.........] - ETA: 0s

InvalidArgumentError:  Input to reshape is a tensor with 1024 values, but the requested shape has 25088
	 [[node model_3/sequential_7/reshape_6/Reshape (defined at <ipython-input-24-5cd3186f7957>:2) ]] [Op:__inference_distributed_function_5139]

Function call stack:
distributed_function


In [20]:
encoder.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 28, 28, 256)       2560      
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 28, 28, 128)       295040    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                802848    
Total params: 1,100,448
Trainable params: 1,100,448
Non-trainable params: 0
_________________________________________________________________


In [25]:
decoder.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 56, 56, 64)        640       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 56, 56, 256)       295168    
_________________________________________________________________
flatten_4 (Flatten)          (None, 802816)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 784)              

In [173]:
#ENCODER
#kernel_initializer='he_uniform' for relu
inp = Input((56, 56,1))
e = Conv2D(32, (3, 3), strides=1,activation='relu',padding='same')(inp)
e = Conv2D(64, (3, 3), strides=2,activation='relu',padding='same')(e)
e = MaxPool2D((2, 2))(e)
#e = Conv2D(64, (3, 3),strides=2,padding='same', activation='relu')(e)
e = BatchNormalization()(e)
#e = MaxPool2D((2, 2))(e)
#e = Conv2D(64, (3, 3),strides=2, activation='relu',padding='same')(e)
e = AveragePooling2D((2,2))(e)
l = Flatten()(e)
l = Dense(16, activation='relu')(l)
#DECODER
d =Dense(3136, activation='relu')(l)
d = Reshape((7,7,64))(d)
#d = ZeroPadding2D(padding=((0,0),(0,1)))(d)
#d = UpSampling2D()(d)
#d = Reshape((14,14,1))(d)
d = Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
d = Conv2DTranspose(128,(3, 3), strides =2,activation='relu', padding='same')(d)
d = BatchNormalization()(d)
#d = UpSampling2D()(d)
#d = Conv2DTranspose(64,(3, 3), strides=2, activation='relu', padding='same')(d)
#d = BatchNormalization()(d)
d = Conv2DTranspose(64,(3, 3),strides=2, activation='relu', padding='same')(d)
d = Conv2DTranspose(32,(3, 3), activation='relu', padding='same')(d)
#d = Reshape((28,28,1))(d)
#d = Dense(32, activation='relu')(d)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(d)
ae = Model(inp, decoded)
ae.summary()
#Input to reshape is a tensor with 1408 values, but the requested shape has 1344

Model: "model_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_89 (InputLayer)        [(None, 56, 56, 1)]       0         
_________________________________________________________________
conv2d_311 (Conv2D)          (None, 56, 56, 32)        320       
_________________________________________________________________
conv2d_312 (Conv2D)          (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_164 (Bat (None, 14, 14, 64)        256       
_________________________________________________________________
average_pooling2d_34 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_78 (Flatten)         (None, 3136)              0  

In [174]:
# compile it using adam optimizer
ae.compile(optimizer="adam", loss="mse")
#Train it by providing training images
ae.fit(X, X, epochs=20000)

Train on 44 samples
Epoch 1/20000
44/44 [==============================] - 1s 33ms/sample - loss: 27869.3370
Epoch 2/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27793.1662
Epoch 3/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27745.8246
Epoch 4/20000
44/44 [==============================] - 1s 18ms/sample - loss: 27736.1333
Epoch 5/20000
44/44 [==============================] - 1s 16ms/sample - loss: 27735.0570
Epoch 6/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27734.8970
Epoch 7/20000
44/44 [==============================] - 1s 14ms/sample - loss: 27734.8572
Epoch 8/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27734.8494
Epoch 9/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27734.8487
Epoch 10/20000
44/44 [==============================] - 1s 18ms/sample - loss: 27734.8549
Epoch 11/20000
44/44 [==============================] - 1s 15ms/sample - loss: 27734.8514

KeyboardInterrupt: 

In [122]:
array_to_img(X[3])

In [1]:
pwd

'C:\\Users\\dedunn\\estee_lauder'

In [71]:
from tensorflow.keras.models import load_model

In [72]:
from tensorflow.keras.models import load_model

In [73]:
model=load_model(r"C:\Users\dedunn\Desktop\weights-val_74-0.98.hdf5")

In [74]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      2432      
_________________________________________________________________
dropout (Dropout)            (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 221, 221, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 221, 221, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 221, 221, 32)      128       
_________________________________________________________________
average_pooling2d (AveragePo (None, 110, 110, 32)      0

In [ ]:
model=Sequential()
model.add(tensorflow.keras.layers.Conv2D(input_shape=(224,224,3),filters=32, kernel_size=(5,5),activation="relu", padding="same")),
model.add(tensorflow.keras.layers.Dropout(0.5)),
model.add(tensorflow.keras.layers.BatchNormalization(axis=-1)),
#model.add(tensorflow.keras.layers.Activation("relu")),
model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=(1,1), padding='valid')),
#model.add(tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same",activation="relu")),
#model.add(tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same",activation="relu")),
#model.add(tensorflow.keras.layers.Dropout(0.5)),
#model.add(tensorflow.keras.layers.BatchNormalization(axis=-1)),
#model.add(tensorflow.keras.layers.Activation("relu")),
#model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(4, 4), strides=(1,1), padding='valid')),
#model.add(tensorflow.keras.layers.Dropout(0.5)),
#model.add(tensorflow.keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding="same",activation="relu")),
model.add(tensorflow.keras.layers.BatchNormalization(axis=-1)),
#model.add(tensorflow.keras.layers.Activation("relu")),
#model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='valid')),
#model.add(tensorflow.keras.layers.AveragePooling2D(pool_size=(4, 4), strides=(1,1), padding='valid')),
#model.add(tensorflow.keras.layers.Dropout(0.5)),
#model.add(tensorflow.keras.layers.Conv2D(filters=224, kernel_size=(3,3), padding="same", activation="relu")),
#model.add(tensorflow.keras.layers.Dropout(0.5)),
#model.add(tensorflow.keras.layers.BatchNormalization(axis=-1)),
#model.add(tensorflow.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='valid')),
model.add(tensorflow.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)),
model.add(tensorflow.keras.layers.Flatten()),
model.add(tensorflow.keras.layers.Dense(44, activation='softmax')),